In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy import optimize

from hyperopt import hp, tpe, Trials, fmin
import hyperopt

In [4]:
# helper functions

X_train = []
y_train = []

def objective(a0, a1, w, f):
    return np.mean((a0+ a1 * np.sin(w*X_train + f) - y_train)**2)

def objective2(args):
    return objective(*args)

def get_hyperopt_params(x, y):
    X_train = x
    y_train = y

    space = [hp.uniform('a0', -1000, 1000),
         hp.uniform('a1', 100, 800), 
         hp.uniform('w', 0, 10), 
         hp.uniform('f', 0, 2 * np.pi)]

    tpe_algo = tpe.suggest
    print (tpe_algo)
    tpe_trials = Trials()

    tpe_best = fmin(fn=objective2, space=space, algo=tpe_algo, trials=tpe_trials, max_evals=500)

    amplitude = float('{a1:.2f}'.format(**tpe_best))
    constant = float('{a0:.2f}'.format(**tpe_best))
    horiz_scale = float('{w:.2f}'.format(**tpe_best))
    offset = float('{f:.2f}'.format(**tpe_best))

    losses = tpe_trials.losses()

    return [amplitude, horiz_scale, offset, constant, min(losses)]

def get_best_hyperopt_params(x, y):
    X_train = x
    y_train = y

    min_loss = -1
    best_amp = 0
    best_w = 0
    best_offset = 0
    best_const  = 0

    for i in range(5):
        ret = get_hyperopt_params(x, y)
        if (ret[4] < min_loss or min_loss < 0):
            best_amp, best_w, best_offset, best_const, min_loss = ret
    
    return [best_amp, best_w, best_offset, best_const]

def sinusoid(x, a, b, c, d):
    return a * np.sin(b * x + c) + d

def scipy_opt(hyperopt_params):
    print('NOT IMPLEMENTED')
    params, params_covariance = optimize.curve_fit(sinusoid, X_train, y_train, p0=hyperopt_params, method='lm')
    return params

def regress_sine_wave(x, y):
    X_train = x
    y_train = y
    hp_params = get_best_hyperopt_params(x, y)
    final_params = scipy_opt(hp_params)
    return final_params

def swap_lr(series):
    for l_key in series:
        if (l_key[0] == 'l'):
            r_key = 'right' + l_key[4:]
            temp = series[l_key]
            series[l_key] = series[r_key]
            series[r_key] = temp
    return series


In [ ]:
''' LAYOUT OF TEMPORAL OPTIMIZATION PIPELINE

define lists for offset and period
define objective functions for left and right

[1] find correct sides, offsets, and periods

for each scale normalized series:

    load the series

    regress left and right heel_y's:
        5 iterations of hyperopt, store params of the one with lowest best loss
        run scipy with hyperopt lowest loss as init guess

    display in subplot
    prompt for input (g or b or s) - good, bad, or skip
    if g, save to offset, side, and period lists, continue
        save period as 2pi / omega
        save offset as (0.5 * pi / omega) - (phi/b)
            add period until offset is greater than 0
        choose side with lower offset
    
    if b, try again
    if s, skip it and make a note of the number skipped for removal

[2] trim examples

find minimum period

for each scale normalized file:
    if foot and entryside mismatch:
        swap all L-R coords (helper function)

    find number of examples: min (((total frames - offset) / period), 3)
    find step size: current_period / min_period

    define a new series and step through at the step size, picking values to put in the normalized list

    for each number of examples:
        define a new series and append all the stepped frame values

'''